# Day 14
Find the description of the problem [here](https://adventofcode.com/2024/day/14)!

## Part 1

Puzzle input:

In [223]:
with open("input_files/day_14.txt") as input_file:
    input = input_file.read()
width = 101
height = 103

Test input:

In [224]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """p=0,4 v=3,-3
# p=6,3 v=-1,-3
# p=10,3 v=-1,2
# p=2,0 v=2,-1
# p=0,0 v=1,3
# p=3,0 v=-2,-2
# p=7,6 v=-1,-3
# p=3,0 v=-1,-2
# p=9,3 v=2,3
# p=7,3 v=-1,2
# p=2,4 v=2,-3
# p=9,5 v=-3,-3"""
# width = 11
# height = 7

Parse the input:

In [225]:
import re

initial_robots = []
for line in input.split("\n"):
    x, y, vx, vy = re.findall(r"-?\d+", line)
    position = (int(x), int(y))
    velocity = (int(vx), int(vy))
    initial_robots.append([position, velocity])

In [226]:
seconds = 100

# Simulate the robots' movement
robots = [line[:] for line in initial_robots]
for i in range(seconds):
    for id, robot in enumerate(robots):
        x, y = robot[0]
        vx, vy = robot[1]

        x += vx
        if x >= width:
            x -= width
        elif x < 0:
            x += width
        y += vy
        if y >= height:
            y -= height
        elif y < 0:
            y += height
        robots[id][0] = (x, y)
        
# Draw the area map with the robot counts
area = [[0 for _ in range(width)] for _ in range(height)]
for robot in robots:
    x, y = robot[0]
    area[y][x] += 1

# Separate into quadrants
ul_quadrant = []
ur_quadrant = []
bl_quadrant = []
br_quadrant = []
for row, line in enumerate(area):
    if row < height // 2:
        ul_quadrant.append(line[:width // 2])
        ur_quadrant.append(line[width // 2 + 1:])
    elif row > height // 2:
        bl_quadrant.append(line[:width // 2])
        br_quadrant.append(line[width // 2 + 1:])

# Count robots in each quadrant
ul_count = 0
for line in ul_quadrant:
    for robots in line:
        ul_count += robots
ur_count = 0
for line in ur_quadrant:
    for robots in line:
        ur_count += robots
bl_count = 0
for line in bl_quadrant:
    for robots in line:
        bl_count += robots
br_count = 0
for line in br_quadrant:
    for robots in line:
        br_count += robots
safety_factor = ul_count * ur_count * bl_count * br_count

print(f"The safety factors after {seconds} seconds is {safety_factor}.")

The safety factors after 100 seconds is 208437768.


## Part 2

This was my first attempt, to try to find a blob of robots that would probably be within the tree. However, after 5 minutes of runtime, simulating 5000 seconds, I decided to stop and try another approach. After getting the solution I tested it and it did indeed work, I just had to wait two more minutes :(

In [227]:
blob = """###
###
###
"""

christmass_tree_found = False
seconds = 0
robots = [line[:] for line in initial_robots]
while not christmass_tree_found:
    for id, robot in enumerate(robots):
        x, y = robot[0]
        vx, vy = robot[1]

        x += vx
        if x >= width:
            x -= width
        elif x < 0:
            x += width
        y += vy
        if y >= height:
            y -= height
        elif y < 0:
            y += height
        robots[id][0] = (x, y)

    area = [["." for _ in range(width)] for _ in range(height)]
    for robot in robots:
        x, y = robot[0]
        area[y][x] = "#"

    # Scan the area for the blob
    for j in range(height - 3 + 1):
        for i in range(width - 3 + 1):
            scan_area = ""
            for r in range(3):
                for c in range(3):
                    scan_area += area[j + r][i + c]
                scan_area += "\n"
            if scan_area == blob:
                christmass_tree_found = True
    
    seconds += 1
    print(f"{seconds} seconds elapsed") if seconds % 100 == 0 else None

print(f"Christmass tree found in {seconds} seconds")

100 seconds elapsed
200 seconds elapsed
300 seconds elapsed
400 seconds elapsed
500 seconds elapsed
600 seconds elapsed
700 seconds elapsed
800 seconds elapsed
900 seconds elapsed
1000 seconds elapsed
1100 seconds elapsed
1200 seconds elapsed
1300 seconds elapsed
1400 seconds elapsed
1500 seconds elapsed
1600 seconds elapsed
1700 seconds elapsed
1800 seconds elapsed
1900 seconds elapsed
2000 seconds elapsed
2100 seconds elapsed
2200 seconds elapsed
2300 seconds elapsed
2400 seconds elapsed
2500 seconds elapsed
2600 seconds elapsed
2700 seconds elapsed
2800 seconds elapsed
2900 seconds elapsed
3000 seconds elapsed
3100 seconds elapsed
3200 seconds elapsed
3300 seconds elapsed
3400 seconds elapsed
3500 seconds elapsed
3600 seconds elapsed
3700 seconds elapsed
3800 seconds elapsed
3900 seconds elapsed
4000 seconds elapsed
4100 seconds elapsed
4200 seconds elapsed
4300 seconds elapsed
4400 seconds elapsed
4500 seconds elapsed
4600 seconds elapsed
4700 seconds elapsed
4800 seconds elapsed
4

Unfortunately I didn't think of the following solution by myself. I unintentionally saw someone do it on Reddit and decided to try it. Just generate images and search for the tree.
Instead of searching through all the images 10000 images, I noticed that counting from 18 seconds, every 101 seconds a pattern appeared, so I only checked at that interval to find the tree.

<img src="day_14_images/18.png">

In [228]:
import numpy as np 
from PIL import Image as img

max_seconds = 10000
robots = [line[:] for line in initial_robots]
for seconds in range(1, max_seconds + 1):
    for id, robot in enumerate(robots):
        x, y = robot[0]
        vx, vy = robot[1]

        x += vx
        if x >= width:
            x -= width
        elif x < 0:
            x += width
        y += vy
        if y >= height:
            y -= height
        elif y < 0:
            y += height
        robots[id][0] = (x, y)

    if (seconds - 18) % 101 == 0:
        area = [[0 for _ in range(width)] for _ in range(height)]
        for robot in robots:
            x, y = robot[0]
            area[y][x] = 255
        area_np = np.array(area, dtype=np.uint8)
        image = img.fromarray(area_np)
        image.save(f"day_14_images/{seconds}.png")

This way, I found the tree by looking at some pictures saved to a folder:

<img src="day_14_images/7492.png">